# Data for Policy 2024 

## Case Study 1: Bayesian Structure Learning with Synthetic Education Data 

In [ ]:

import numpy as np 
import pandas as pd

import networkx as nx
import seaborn as sns

import matplotlib.pyplot as plt

from collections import Counter

import networkx as nx
import matplotlib.pyplot as plt

import pickle

import os 
from scores.bge import BGeScore

from data.datagen import SyntheticDataset
from utils.graph_utils import *

import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

from inference.posterior import *
from collections import Counter, defaultdict

from evaluation.metrics import *
sns.set_style("whitegrid", {'axes.grid' : False})


In this example, we will consider the following variables: LowIncome, SingleParent, MentalHealth, NotCompletedYr12, Unemployed. 

Variable descriptions are as follows:

- LowIncome: Percentage of low income households in the area
- SingleParent: Percentage of single parent households in the area
- MentalHealth: Percentage of people with mental health issues in the area
- NotCompletedYr12: Percentage of people who did not complete year 12 in the area
- Unemployed: Percentage of people who are unemployed in the area


### Defining the True Data Generation Process

In [ ]:
# specifying the true dag

G, positions = build_ground_truth_graph()

plot_graph( G, positions, figsize=(7, 5), save = True , filepath= "./results/true_dag.png")

In [ ]:

nodes = ["Unemployed", "LowIncome", "NotCompletedYr12", "SingleParent", "MentalHealth"]
num_nodes = len(nodes)
num_obsv = 100
degree = 2
noise_scale = 0.5

sparse_adj_matrix = nx.adjacency_matrix(G)
dense_adj_matrix = sparse_adj_matrix.todense()

np.random.seed(794) # for reproducibility | 206 | 267 | 794 | 892

sdj = SyntheticDataset(num_nodes, num_obsv, nodes, degree, noise_scale=noise_scale, true_dag=dense_adj_matrix)

data = sdj.data

In [ ]:
data.to_csv("results/synthetic_data.csv", index=False)

### True Posterior Distribution

In [ ]:

all_dags = generate_all_dags( data, BGeScore, gen_augmented_priors = False)
true_distr = compute_true_distribution( all_dags, with_aug_prior = False )

In [ ]:
save_path = "./results/true_posterior.png"

true_distr_plot, key_to_id, id_to_key = plot_posterior_distribution(true_distr, true_graph = dense_adj_matrix, true_graph_color = "#E74C3C", prob_threshold=0.003, figsize = (14,5), title = "",  save_path=save_path  )


In [ ]:
ID = 15790
key = id_to_key[ ID ]
G_true = generate_graph_from_key( key, nodes)

plot_graph(G_true, pos = positions, figsize=(5, 4), title="", save = True , filepath= f"./results/G_{ID}_True_DAG.png")

In [ ]:
ID = 15717
key = id_to_key[ ID ]
G_ID = generate_graph_from_key( key, nodes)

res = compare_graphs( G_ID, G_true)
edges_to_highlight = res['Reversed Edges'] + res['Added Edges'] + res['Deleted Edges']

plot_graph(G_ID, pos = positions, highlighted_edges=edges_to_highlight, figsize=(5, 4), title="", save = True , filepath= f"./results/G_{ID}.png");


In [ ]:
ID = 15722
key = id_to_key[ ID ]
G_ID = generate_graph_from_key( key, nodes)

res = compare_graphs( G_ID, G_true)
edges_to_highlight = res['Reversed Edges'] + res['Added Edges'] + res['Deleted Edges']

plot_graph(G_ID, pos = positions, highlighted_edges=edges_to_highlight, figsize=(5, 4), title="", save = True , filepath= f"./results/G_{ID}.png");


In [ ]:
ID = 15978
key = id_to_key[ ID ]
G_ID = generate_graph_from_key( key, nodes)

res = compare_graphs( G_ID, G_true)
edges_to_highlight = res['Reversed Edges'] + res['Added Edges'] + res['Deleted Edges']

plot_graph(G_ID, pos = positions, highlighted_edges=edges_to_highlight, figsize=(5, 4), title="", save = True , filepath= f"./results/G_{ID}.png");


In [ ]:
ID = 17336
key = id_to_key[ ID ]
G_ID = generate_graph_from_key( key, nodes)

res = compare_graphs( G_ID, G_true)
edges_to_highlight = res['Reversed Edges'] + res['Added Edges'] + res['Deleted Edges']

plot_graph(G_ID, pos = positions, highlighted_edges=edges_to_highlight, figsize=(5, 4), title="", save = True , filepath= f"./results/G_{ID}.png");


In [ ]:
ID = 15962
key = id_to_key[ ID ]
G_ID = generate_graph_from_key( key, nodes)

res = compare_graphs( G_ID, G_true)
edges_to_highlight = res['Reversed Edges'] + res['Added Edges'] + res['Deleted Edges']

plot_graph(G_ID, pos = positions, highlighted_edges=edges_to_highlight, figsize=(5, 4), title="", save = True , filepath= f"./results/G_{ID}.png");


## Linear Regression

In [ ]:
Glr, poslr = build_linear_regr_graph()

plot_graph( Glr, poslr, figsize=(6, 4), save = True , filepath= "./results/LR_dag.png")

In [ ]:

# Prepare predictors and target variable
X = data.drop(["NotCompletedYr12"], axis=1)
y = data["NotCompletedYr12"]


# Add a constant to the predictors (for the intercept)
X_const = sm.add_constant(X)

# Fit the regression model
model = sm.OLS(y, X_const).fit()

# Print the summary, which includes p-values
print(model.summary())

# Extract the p-values
p_values = model.pvalues

# Consider variables with p < 0.05 as statistically significant
significant_vars = p_values[p_values < 0.05].index.tolist()

# Print the names of significant variables
print("\nSignificant variables:", significant_vars)




In [ ]:
highlighted_edges = []
for var in significant_vars:
    highlighted_edges.extend([(var, "NotCompletedYr12")])
    
plot_graph( Glr, pos = poslr, highlighted_edges = highlighted_edges,  figsize=(6, 4),  save = True , filepath= "./results/true_dag_stat_sig.png")

make an example where we start a BN that is the Linear Regression and then generate data and see if I can recover the GT

## Structure Learning - Finding the Best Model using PC Algorithm

In [ ]:

from pgmpy.utils import get_example_model
from pgmpy.estimators import PC

est = PC(data)
model_pc = est.estimate(ci_test='pearsonr',  return_type='dag', significance_level=0.05, max_cond_vars=2)

# convert dag to graph
pc_dag = nx.DiGraph()
pc_dag.add_edges_from(model_pc.edges())


In [ ]:
plot_graph(pc_dag, figsize=(5, 4),  save = True , filepath= "./results/pc_algo_dag.png" )

In [ ]:

edge_frequency_heatmap( [pc_dag], nodes = list(data.columns), title = "PC Algorithm", figsize=(6, 6), save_path="./results/pc_algo_edge_map.png" )


In [ ]:
edge_frequency_heatmap( [G], nodes = list(data.columns), title = "True Data Generation Graph", figsize=(6, 6), save_path="./results/true_dag_edge_map.png");


## Structure Learning - Partition MCMC

In [ ]:
%load_ext rpy2.ipython

from rpy2.robjects import pandas2ri
pandas2ri.activate()

FLAG = 1 # set to 1 to run PartitionMCMC

In [ ]:
%%R -o graph_list_R -i FLAG

library(BiDAG)
library(readr)
library(pcalg)
library(doParallel)
library(foreach)
library(igraph)


runPartMCMC_original_exp <- function(data_path,maxiter,initial_graph_path,result_path,n_nodes){

    data <- as.data.frame(read_csv(data_path,show_col_types=FALSE))
    #initial_graph <- as.data.frame(read_csv(initial_graph_path,show_col_types=FALSE))

    myScore<-BiDAG::scoreparameters("bge", data)
    partfit<-BiDAG::partitionMCMC(myScore,iterations = maxiter, startspace = matrix(1, n_nodes, n_nodes), stepsave = 1)
    graph_list <- partfit$traceadd$incidence

    graph_list_final <- lapply(graph_list, as.matrix)
    return(graph_list_final)
}

# set seed
set.seed(11)

exp_id <- 1
n_nodes <- 5
max_iter <- 300000
graph_list_R <- list()

my_data_path <- "results/synthetic_data_1.csv"
my_initial_dag_path <- "results/initial_graph.csv"

if( FLAG == 1 ){
    print("FLAG = 1, initiating Partition MCMC")
    graph_list_R <- runPartMCMC_original_exp(my_data_path, max_iter,my_initial_dag_path, "", n_nodes)
} else {
    print("FLAG = 0, skipping Partition MCMC...")
}


In [ ]:
if FLAG == 1:
    # process R objects to python
    graph_list = []
    for i in graph_list_R.items():
        graph_list.append( i[1] )
        
    # delete the first element
    graph_list = graph_list[1:]
    print("Total graphs processed", len(graph_list))
    with open("results/pmcmc_graph_list_1.pkl", "wb") as f:
        pickle.dump( graph_list, f )
else: # if PMCMC was not called, use our pre computed graph_list
    with open("results/pmcmc_graph_list.pkl", "rb") as f:
        graph_list = pickle.load( f )
    print("Total graphs loaded", len(graph_list))

In [ ]:

approx_distr = compute_approx_distribution_index(graph_list, true_distr)


In [ ]:
JSD = jensen_shannon_divergence( approx_distr, true_distr ) 
print("JSD(approx_distr, true_distr) = ", np.round(JSD, 6))

In [ ]:

save_path = "./results/approx_distr.png"
ylabel = r'MCMC Approximate P($G | Data$)'
approx_distr_plot, key_to_id, id_to_key = plot_posterior_distribution(approx_distr, ylabel = ylabel, true_graph = dense_adj_matrix, true_graph_color = "#E74C3C", prob_threshold=0.003, figsize = (14,5), title = "",  save_path=save_path  )


In [ ]:
ID = 15790
key = id_to_key[ ID ]
key

In [ ]:
# get the top N entries of the approximate distribution
N =10
approx_distr_rounded = {key: round(value, 4) for key, value in approx_distr.items()}
top_n_entries = dict(sorted(approx_distr_rounded.items(), key=lambda item: (-item[1], item[0]))[:N])
top_n_entries


In [ ]:
# ... and their corresponding IDs
ids = [ key_to_id[k] for k, v in top_n_entries.items()]
ids

In [ ]:
def plot_graph(G : nx.DiGraph, pos = None, highlighted_edges = None, title=None, figsize = (5,3), node_size=2000, node_color="skyblue", k=5, save = False, filepath = None):
    
    if pos is None:
        pos = nx.spring_layout(G, k=k)

    plt.figure(figsize=figsize)
    
    
    nx.draw(G, with_labels=True, arrowsize=20, arrows=True, node_size=node_size, node_color=node_color, pos=pos)
    if highlighted_edges:
        nx.draw(G, edgelist=highlighted_edges,edge_color='red',width=5, arrowsize=25, alpha=0.4, with_labels=True, arrows=True, node_size=node_size, node_color=node_color, pos=pos)
    
    plt.gca().margins(0.20)
    
    if title:
        plt.title(title)
    plt.axis("off")
    
    if save:
        #plt.tight_layout()
        plt.savefig(filepath, dpi=300)
    plt.show()

In [ ]:
ID = 17336
key = id_to_key[ ID ]
G_ID = generate_graph_from_key( key, nodes)

res = compare_graphs( G_ID, G_true)
edges_to_highlight = res['Reversed Edges'] + res['Added Edges'] + res['Deleted Edges']

plot_graph(G_ID, pos = positions, highlighted_edges=edges_to_highlight, figsize=(5, 4), title=fr"Predicted DAG with Pr($G_{{{ID}}}$ | D) = ${top_n_entries[key]}$", save = True , filepath= f"./results/G_{ID}.png");


In [ ]:
ID = 15978
key = id_to_key[ ID ]
G_ID = generate_graph_from_key( key, nodes)

res = compare_graphs( G_ID, G_true)
edges_to_highlight = res['Reversed Edges'] + res['Added Edges'] + res['Deleted Edges']

plot_graph(G_ID, pos = positions, highlighted_edges=edges_to_highlight, figsize=(5, 4), title=fr"Predicted DAG with Pr($G_{{{ID}}}$ | D) = ${top_n_entries[key]}$", save = True , filepath= f"./results/G_{ID}.png");


In [ ]:
ID = 13578
key = id_to_key[ ID ]
G_ID = generate_graph_from_key( key, nodes)

res = compare_graphs( G_ID, G_true)
edges_to_highlight = res['Reversed Edges'] + res['Added Edges'] + res['Deleted Edges']

plot_graph(G_ID, pos = positions, highlighted_edges=edges_to_highlight, figsize=(5, 4), title=fr"Predicted DAG with Pr($G_{{{ID}}}$ | D) = ${top_n_entries[key]}$", save = True , filepath= f"./results/G_{ID}.png");


In [ ]:
ID = 15790
key = id_to_key[ ID ]
G_ID = generate_graph_from_key( key, nodes)

res = compare_graphs( G_ID, G_true)
edges_to_highlight = res['Reversed Edges'] + res['Added Edges'] + res['Deleted Edges']

plot_graph(G_ID, pos = positions, highlighted_edges=edges_to_highlight, figsize=(5, 4), title=fr"[True DAG] Predicted DAG with Pr($G_{{{ID}}}$ | D) = ${top_n_entries[key]}$", save = True , filepath= f"./results/G_{ID}.png");


In [ ]:
ID = 18048
key = id_to_key[ ID ]
G_ID = generate_graph_from_key( key, nodes)

res = compare_graphs( G_ID, G_true)
edges_to_highlight = res['Reversed Edges'] + res['Added Edges'] + res['Deleted Edges']

plot_graph(G_ID, pos = positions, highlighted_edges=edges_to_highlight, figsize=(5, 4), title=fr"Predicted DAG with Pr($G_{{{ID}}}$ | D) = ${top_n_entries[key]}$", save = True , filepath= f"./results/G_{ID}.png");


In [ ]:
edge_frequency_heatmap_np( graph_list, nodes = list(data.columns), figsize=(6, 6), title = "Edge Occurrence Probabilities in Partition MCMC", save_path= "./results/pmcmc_edge_map.png")


In [ ]:
# perform model Bayesian average to compute the "best" graph from the distribution
prob_matrix = graph_model_averaging( graph_list, 5, prob = 0.5)
plot_graph_from_adj_mat(prob_matrix, node_labels = nodes, pos = positions, figsize=(5, 4), title="", save = True , filepath= "./results/mba_dag.png")

In [ ]:
edge_frequency_heatmap_np( [prob_matrix], nodes = list(data.columns), figsize=(6, 5), title = "Edge Occurrence Probabilities in Partition MCMC")


### Plot Curves

In [ ]:
import seaborn as sns
sns.set()

In [ ]:
expr_post = mcmc_post_process(5, 5)

In [ ]:
plot_mcmc_metric(5, "./results", title="Jenson-Shannon Divergence for Partition MCMC")


In [ ]:
with open("./results/true_distr.pkl", 'rb') as f:
    true_distr = pickle.load(f)

with open("./results/pmcmc_graph_list.pkl", 'rb') as f:
    graph_list = pickle.load(f)

In [ ]:
# sort the true distribution
true_distr = dict(sorted(true_distr.items(), key=lambda item: item[1], reverse=True))

approx_distr = compute_approx_distribution_index(graph_list, true_distr)

In [ ]:


plot_approx_posterior_distribution(true_distr, true_graph=dense_adj_matrix, prob_threshold=0.001, figsize=(12, 7), title="PMCMC Approximate Posterior Distribution", algo1_scores=approx_distr, label1="PMCMC");
